In [1]:
from utils.config_utils import parse_config
parsed_config = parse_config('./configs/helsinki-taboeta-de-en.yml', pretrained=True)

Using custom data configuration de-en-lang1=de,lang2=en
Reusing dataset tatoeba (C:\Users\gerso\.cache\huggingface\datasets\tatoeba\de-en-lang1=de,lang2=en\0.0.0\b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6)


  0%|          | 0/1 [00:00<?, ?it/s]

[276634, 20738, 2500, 2500, 5000]


In [2]:
import time
import numpy as np
from collections import Counter
model =parsed_config["model"].to("cuda")
model.eval()
tokenizer = parsed_config["tokenizer"]
example_sentence = "wie gehts"#"Es ist schwierig, objektiv über Gold zu sprechen"
n_samples = 10

In [3]:
print(model)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58101, 512, padding_idx=58100)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58101, 512, padding_idx=58100)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [4]:
from utils.translation_model_utils import translate
import torch

In [11]:
# Measure running time of 
start_time = time.time()
example_sentence_repeated = [example_sentence] *1000
translations = translate(model, tokenizer, example_sentence_repeated, batch_size=8,)
print(Counter(translations))
print("--- %s seconds ---" % (time.time() - start_time))

Counter({"We're going.": 32, 'Please go.': 17, "We're doing.": 13, 'How are things going.': 13, "You're going.": 9, 'Like you go.': 8, "We're fine.": 8, "We'll go.": 7, "We're going the way.": 7, "We're walking.": 7, "There's no way to go.": 7, "We're as going.": 7, "You're as going.": 6, "You're doing.": 6, 'How are we going.': 6, "We're going!": 5, 'As he goes.': 5, "As you're going.": 5, "We're like going.": 5, "We're dying.": 5, "There's no way.": 5, "We're nearly going.": 5, "We're leaving.": 5, 'How are things going?': 4, "We'll go the way.": 4, "We'll go!": 4, 'How are things going!': 4, 'We go the way.': 4, "We're doing!": 4, "That's how to go.": 4, "It's like going.": 4, 'As you go.': 4, "We're sinking.": 3, "We're How going.": 3, "There's nothing like going.": 3, 'How are things?': 3, "We're up to things.": 3, "There's a way.": 3, "There's like going.": 3, 'Just like done.': 3, 'As it goes.': 3, 'Like it goes.': 3, 'How are you going.': 3, 'As to go.': 3, 'This is how we go.'

In [6]:
# tokenized_samples = tokenizer(example_sentence, return_tensors="pt").to("cuda")
# start_time = time.time()

# sample = model.generate(
#             **tokenized_samples,
#             do_sample=True,
#             top_k=0,
#             temperature=1.0,
#             output_scores=True,
#             return_dict_in_generate=True,
#             num_beams=1
    
#         )
# print(sample)
# print("--- %s seconds ---" % (time.time() - start_time))

In [7]:
### We will try to speed this up.
# First we will wrap the model



class ModelWrapperForFastSampling:
    
    def __init__(self, model, tokenizer):
        self.model = model
        
        self.encoder = model.model.encoder
        self.decoder = model.model.decoder
        self.head = model.lm_head
        self.tokenizer = tokenizer
        

    
    def generate_samples(self, sentence, n_samples=1000):
        print(dir(self.model))
    
        possible_tokens = np.arange(self.tokenizer.vocab_size)
        start_token = self.tokenizer.pad_token_id
        print(start_token)
        with torch.no_grad():
            tokenized_samples = self.tokenizer(sentence, return_tensors="pt").to("cuda")
            print(tokenized_samples)
            
            temp = self.model.forward(**tokenized_samples, decoder_input_ids=torch.tensor([[start_token]]).to("cuda"))
            temp_logits = temp["logits"]
            
            start_time = time.time()
            hidden_state = self.encoder.forward(**tokenized_samples)["last_hidden_state"]
            tokens = torch.Tensor([[start_token]]).to("cuda").long()
        
            start_time = time.time()
            out = self.decoder(tokens, encoder_hidden_states=hidden_state)
            
            logits = self.head(out.last_hidden_state)
            probs = torch.softmax(logits, dim=-1, ).cpu().flatten().numpy()
            # Normalize
            print(np.max(probs))

            samples = np.random.choice(len(probs),  size=n_samples, p=probs,replace=True)
            print(Counter(samples))
            
            print(torch.all(logits == temp_logits))
            print(logits.shape)
            logits[:,:, self.tokenizer.pad_token_id] = float("-inf")
            print("bias")
            print(torch.max(model.final_logits_bias))
            print(model.final_logits_bias)
            probs = torch.softmax(logits, dim=-1,).cpu().flatten().numpy()
            print(np.max(probs))
           

In [8]:
wrapped_model = ModelWrapperForFastSampling(model, tokenizer)

wrapped_model.generate_samples(example_sentence)

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_compatibility_gradient_checkpointing', '_backward_hooks', '_buffers', '_call_impl', '_convert_head_mask_to_5d', '_create_or_get_repo', '_expand_inputs_for_generation', '_forward_hooks', '_forward_pre_hooks', '_from_config', '_get_backward_hooks', '_get_decoder_start_token_id', '_get_logits_processor', '_get_logits_warper', '_get_name', '_get_pad_token_id', '_get_repo_url_from_name', '_get_resized_embeddings', '_get_resized_lm_head', '_get_stopping_criteria', '_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_init_weights', '_is_full_

In [9]:
tokenizer.decode([0])

''

In [10]:
print(tokenizer.pad_token_id)

58100
